In [1]:
# Import libraries and packages for the project 

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from time import sleep
import csv

print('- Finish importing packages')

- Finish importing packages


In [2]:
# Task 1: Login to Linkedin

# Task 1.1: Open Chrome and Access Linkedin login site
driver = webdriver.Chrome()
sleep(2)
url = 'https://www.linkedin.com/login'
driver.get(url)
print('- Finish initializing a driver')
sleep(2)

# Task 1.2: Import username and password
credential = open('credentials.txt')
line = credential.readlines()
username = line[0]
password = line[1]
print('- Finish importing the login credentials')
sleep(2)

# Task 1.2: Key in login credentials
email_field = driver.find_element_by_id('username')
email_field.send_keys(username)
print('- Finish keying in email')
sleep(3)

password_field = driver.find_element_by_name('session_password')
password_field.send_keys(password)
print('- Finish keying in password')
sleep(2)

# Task 1.2: Click the Login button
signin_field = driver.find_element_by_xpath('//*[@id="app__container"]/main/div[2]/form/div[3]/button')
signin_field.click()
sleep(3)

print('- Finish Task 1: Login to Linkedin')

- Finish initializing a driver
- Finish importing the login credentials
- Finish keying in email
- Finish keying in password
- Finish Task 1: Login to Linkedin


In [3]:
# Task 2: Search for the profile we want to crawl

# Task 2.1: Locate the search bar element
search_field = driver.find_element_by_xpath('//*[@id="ember20"]/input')

# Task 2.2: Input the search query to the search bar
search_query = input('What profile do you want to scrape? ')
search_field.send_keys(search_query)

# Task 2.3: Search
search_field.send_keys(Keys.RETURN)

print('- Finish Task 2: Search for profiles')

- Finish Task 2: Search for profiles


In [13]:
search_url = "https://www.linkedin.com/search/results/people/?facetSchool=%5B%22194005%22%2C%22370029%22%2C%223153429%22%2C%223219010%22%2C%224143035%22%2C%22453161%22%5D&keywords=full%20stack%20developer&origin=FACETED_SEARCH&page=70"

driver.get(search_url)

In [4]:
# Task 3: Scrape the URLs of the profiles

# Task 3.1: Write a function to extract the URLs of one page
def GetURL():
    page_source = BeautifulSoup(driver.page_source)
    profiles = page_source.find_all('a', class_ = 'search-result__result-link ember-view')
    all_profile_URL = []
    for profile in profiles:
        profile_ID = profile.get('href')
        profile_URL = "https://www.linkedin.com" + profile_ID
        if profile_URL not in all_profile_URL:
            all_profile_URL.append(profile_URL)
    return all_profile_URL

# Task 3.2: Navigate through many page, and extract the profile URLs of each page

input_page = int(input('How many pages you want to scrape: '))
URLs_all_page = []
for page in range(input_page):
    URLs_one_page = GetURL()
    sleep(2)
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);') #scroll to the end of the page
    sleep(3)
    next_button = driver.find_element_by_class_name("artdeco-pagination__button--next")
    driver.execute_script("arguments[0].click();", next_button)
    URLs_all_page = URLs_all_page + URLs_one_page
    sleep(2)

print('- Finish Task 3: Scrape the URLs of ', len(URLs_all_page), ' profiles')

- Finish Task 3: Scrape the URLs of  4  profiles


In [11]:
# Task 4: Scrape the data of 1 Linkedin profile, and write the data to a .CSV file

driver.get(URLs_all_page[0])
page_source = BeautifulSoup(driver.page_source, "html.parser")
print(page_source)

me="url_card_action_click_highlights_shared_connections" href="https://www.linkedin.com/search/results/people/?facetNetwork=%5B%22F%22%5D&amp;facetConnectionOf=%5B%22ACoAAC51I4QBAHg7rUQ-y3mrocbvac5Jb5mhsLE%22%5D&amp;origin=MEMBER_PROFILE_CANNED_SEARCH&amp;RESULT_TYPE=PEOPLE" id="ember179"> <h3 class="pv-highlight-entity__primary-text t-16 t-black t-bold">
            1 mutual connection
          </h3>
<p class="pv-highlight-entity__secondary-text t-14 t-black--light t-normal">
              You and Tai both know Gia Bảo Thái
            </p>
</a><!-- --> </div>
</div>
</div>
</li>
</ul>
<!-- --></section>
</div>
</div>
<div class="pv-oc ember-view" id="ember80"> <section class="artdeco-container-card pv-profile-section pv-about-section artdeco-card ember-view" id="ember124" style="display: none;"><header class="pv-profile-section__card-header">
<h2 class="pv-profile-section__card-heading">
    About
  </h2>
<!-- --></header>
<!-- --></section>
</div>
<!-- -->
<div class="pv-deferred-a

In [22]:
info_div = page_source.find('div', class_ = "flex-1 mr5")
info_loc = info_div.find_all('ul')

name = info_loc[0].find('li').get_text().strip()
print('Profile name is: ', name)

location = info_loc[1].find('li').get_text().strip()
print('Profile location is: ', location)

title = info_div.find('h2').get_text().strip()
print('Profile title is: ', title)

Profile name is:  Tai DuongQuoc
Profile location is:  Vietnam
Profile title is:  Software Engineer at FPT Telecom Big Data


In [7]:
# Task 4: Scrape the data of 1 Linkedin profile, and write the data to a .CSV file

with open('output8.csv', 'w',  newline = '') as file_output:
    headers = ['Name', 'Job Title', 'Location', 'URL']
    writer = csv.DictWriter(file_output, delimiter=',', lineterminator='\n',fieldnames=headers)
    writer.writeheader()
    for linkedin_URL in URLs_all_page:
        driver.get(linkedin_URL)
        print('- Accessing profile: ', linkedin_URL)

        page_source = BeautifulSoup(driver.page_source, "html.parser")

        info_div = page_source.find('div',{'class':'flex-1 mr5'})
        info_loc = info_div.find_all('ul')
        name = info_loc[0].find('li').get_text().strip() #Remove unnecessary characters 
        print('--- Profile name is: ', name)
        location = info_loc[1].find('li').get_text().strip() #Remove unnecessary characters 
        print('--- Profile location is: ', location)
        title = info_div.find('h2').get_text().strip()
        print('--- Profile title is: ', title)
        writer.writerow({headers[0]:name, headers[1]:location, headers[2]:title, headers[3]:linkedin_URL})
        print('\n')

print('Mission Completed!')

- Accessing profile:  https://www.linkedin.com/in/tai-duongquoc-370042199/
--- Profile name is:  Tai DuongQuoc
--- Profile location is:  Vietnam
--- Profile title is:  Software Engineer at FPT Telecom Big Data


- Accessing profile:  https://www.linkedin.com/in/kim-tri-huynh-27676518b/
--- Profile name is:  Kim Tri Huynh
--- Profile location is:  District 9, Ho Chi Minh City, Vietnam
--- Profile title is:  Full Stack Engineer at FPT Software


- Accessing profile:  https://www.linkedin.com/in/tanng18011997/
--- Profile name is:  Tân Ngô
--- Profile location is:  District 12, Ho Chi Minh City, Vietnam
--- Profile title is:  Software Engineer at ASCVN Education Solution


- Accessing profile:  https://www.linkedin.com/in/tu-bui-743a59166/
--- Profile name is:  Tu Bui
--- Profile location is:  Vietnam
--- Profile title is:  IT Engineer at Sofiamedix Software Co.,Ltd


Mission Completed!


In [5]:
# Task 3: Scrape the URLs of the profiles

# Task 3.1: Write a function to extract the URLs of one page
def GetURL():
    page_source = BeautifulSoup(driver.page_source)
    profiles = page_source.find_all('a', class_ = 'search-result__result-link ember-view')
    all_profile_URL = []
    for profile in profiles:
        profile_ID = profile.get('href')
        profile_URL = "https://www.linkedin.com" + profile_ID
        if profile_URL not in all_profile_URL:
            all_profile_URL.append(profile_URL)
    return all_profile_URL

print(GetURL())

['https://www.linkedin.com/in/nguyen-n-95256b166/', 'https://www.linkedin.com/in/anh-le-andy/', 'https://www.linkedin.com/in/khoango125/', 'https://www.linkedin.com/in/tanng18011997/']


In [6]:
URLs = GetURL()
for i in URLs:
    print(i)

https://www.linkedin.com/in/nguyen-n-95256b166/
https://www.linkedin.com/in/anh-le-andy/
https://www.linkedin.com/in/khoango125/
https://www.linkedin.com/in/tanng18011997/
